`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
`Семестр: осень 2022/2023`   

# Лабораторная работа №6: Машины опорных векторов  

В практических примерах ниже показано:   

* как классифицировать данные с помощью модели SVM
* как использовать конвейеры для подгонки модели и применения её к новым данным 

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.  

*Модели*: SVM  
*Данные*:`wdbc.data (Breast Cancer Wisconsin)`. Источник: [сайт Калифорнийского университета в Ирвине]

# Указания к выполнению


## Загружаем пакеты

In [1]:
# загрузка пакетов: инструменты -------------------------------------------- 
#  работа с массивами

import numpy as np 
#  фреймы данных
import pandas as pd 
#  графики 
import matplotlib as mpl 
#  стили и шаблоны графиков на основе matplotlib 
import seaborn as sns 
# перекодировка символьных показателей 
from sklearn.preprocessing import LabelEncoder
#  для таймера 
import time 
 
# загрузка пакетов: модели ------------------------------------------------- 
#  SVM 
from sklearn.svm import SVC 
# логистическая рагрессия 
from sklearn.linear_model import LogisticRegression 
# стандартизация
from sklearn.preprocessing import StandardScaler 
# метод главных компонент 
from sklearn.decomposition import PCA 
# конвейеры
from sklearn.pipeline import make_pipeline 
# перекрёстная проверка и метод проверочной выборки 
from sklearn.model_selection import cross_val_score, train_test_split 
# для перекрёстной проверки и сеточного поиска 
from sklearn.model_selection import KFold, GridSearchCV 
#  сводка по точности классификации 
from sklearn.metrics import classification_report 

In [2]:
# константы
#  ядро для генератора случайных чисел
my_seed = 13
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
sns.color_palette("Set2")

[(0.4, 0.7607843137254902, 0.6470588235294118),
 (0.9882352941176471, 0.5529411764705883, 0.3843137254901961),
 (0.5529411764705883, 0.6274509803921569, 0.796078431372549),
 (0.9058823529411765, 0.5411764705882353, 0.7647058823529411),
 (0.6509803921568628, 0.8470588235294118, 0.32941176470588235),
 (1.0, 0.8509803921568627, 0.1843137254901961),
 (0.8980392156862745, 0.7686274509803922, 0.5803921568627451),
 (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]

## Загружаем данные

Набор данных можно загрузить напрямую по ссылке: <https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv>.Справочник к данным доступен по адресу: <https://github.com/aksyuk/MTML/blob/main/Labs/data/CodeBook_default_of_credit_card_clients.md>. 
Загружаем данные во фрейм и выясняем их размерность. В таблице много строк, поэтому для экономии времени загрузку сделаем в два шага: сначала скачаем таблицу и сохраним в папку `'./data'`, затем прочитаем её во фрейм. Перед скачиванием проверим, нет ли уже такого файла в папке с данными.  

In [3]:
# загружаем таблицу и превращаем её во фрейм
fileURL = 'https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv'
DF_raw = pd.read_csv(fileURL)

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (30000, 24)


In [4]:
# первые строки
DF_raw.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


Все столбцы количественные.  

In [5]:
# типы столбцов 
DF_raw.dtypes

LIMIT_BAL    int64
SEX          int64
EDUCATION    int64
MARRIAGE     int64
AGE          int64
PAY_0        int64
PAY_2        int64
PAY_3        int64
PAY_4        int64
PAY_5        int64
PAY_6        int64
BILL_AMT1    int64
BILL_AMT2    int64
BILL_AMT3    int64
BILL_AMT4    int64
BILL_AMT5    int64
BILL_AMT6    int64
PAY_AMT1     int64
PAY_AMT2     int64
PAY_AMT3     int64
PAY_AMT4     int64
PAY_AMT5     int64
PAY_AMT6     int64
Y            int64
dtype: object

Отложим 10% наблюдений для прогноза.

In [6]:
# наблюдения для моделирования 
DF = DF_raw.sample(frac=0.9, random_state=my_seed) 
# отложенные наблюдения 
DF_predict = DF_raw.drop(DF.index) 

# Преобразование исходных данных и построение моделей  

Как указано в справочнике к данным, объясняющие переменные уже являются производными от описательных статистик 10 показателей, рассчитанных по визуализации клеток опухоли. Все эти переменные количественные, и как видно из отчёта ниже, их значения неотрицательные, а разброс различается.   

In [7]:
 DF.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
count,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000,...,27000.000000,27000.000000,27000.000000,27000.000000,2.700000e+04,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000
mean,167725.543704,1.603963,1.853741,1.550926,35.474556,-0.018000,-0.135185,-0.167296,-0.223778,-0.269963,...,43080.955222,40127.070519,38698.920593,5683.113296,5.870060e+03,5210.406111,4846.425593,4825.904481,5231.434222,0.219556
std,129987.024498,0.489081,0.790828,0.522113,9.215236,1.125768,1.198147,1.197849,1.170744,1.135529,...,64075.202730,60559.110748,59274.581650,16555.560844,2.219613e+04,17518.374265,15846.384168,15359.625271,17838.849871,0.413953
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2294.000000,1737.250000,1237.250000,1000.000000,8.360000e+02,390.000000,300.000000,237.750000,115.000000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19000.000000,18065.500000,16979.000000,2110.000000,2.007000e+03,1816.500000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,54562.500000,50080.750000,49107.000000,5009.250000,5.000000e+03,4512.000000,4027.000000,4040.250000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000


В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и
логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент.  

## Стандартизация и переход к главным компонентам

In [8]:
# стандартизация
sc = StandardScaler() 
X_train_std = sc.fit_transform(DF.iloc[:, 0:23].values) 
 
# оцениваем объяснённую главными компонентами дисперсию 
pca = PCA() 
X_train_pca = pca.fit_transform(X_train_std) 
 
# считаем доли объяснённой дисперсии 
frac_var_expl = pca.explained_variance_ratio_ 
print('Доли объяснённой дисперсии по компонентам в PLS:\n', 
     np.around(frac_var_expl, 3), 
     '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.284 0.178 0.067 0.064 0.045 0.041 0.04  0.039 0.037 0.034 0.033 0.03
 0.025 0.023 0.017 0.011 0.011 0.008 0.006 0.003 0.002 0.001 0.001] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 46.2% разброса 23 объясняющих переменных. 
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline [https://scikit-learn.ru/6-1-pipelines-and-composite-estimators/] и оценим точность логистической регрессии с помощью перекрёстной проверки. конвейеры позволяет последовательно применять описанные преобразования. 

## Модель логистической регрессии с перекрёстной проверкой

In [9]:
# данные для обучения моделей
X_train = DF.iloc[:, 0:23] 
y_train = DF.iloc[:, -1] 
 
# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит 
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=2), 
                        LogisticRegression(random_state=my_seed, solver='lbfgs'))
 
# будем сохранять точность моделей в один массив: 
score = []
score_models = list() 
 
# считаем точность с перекрёстной проверкой, показатель Acc 
cv = cross_val_score(estimator=pipe_lr, X=X_train, y=y_train, cv=5, 
                     scoring='accuracy') 
 
# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('sc_pca_logit') 
 
print('Acc с перекрёстной проверкой', 
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.798


##  SVM с перекрёстной проверкой

Построим несколько вариантов модели SVM с различными ядерными функциями.

In [10]:
pipe_svc = make_pipeline(StandardScaler(), 
                         SVC(random_state = my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1] 
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, 
                  scoring = 'accuracy', refit = True, cv = kfold, 
                  n_jobs = -1)

# таймер
tic = time.perf_counter()

# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)

# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')


Сеточный поиск занял 2040.93 секунд


In [11]:
# точность лучшей модели 
np.around(gs.best_score_, 3)

0.816

In [12]:
# параметры лучшей модели 
#  * ядерная функция 
gs.best_estimator_.get_params()['svc__kernel']

'rbf'

In [13]:
# * параметр регуляризации 
gs.best_estimator_.get_params()['svc__C']

0.1

In [14]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid') 
gs.best_estimator_.get_params()['svc__gamma'] 

0.1

In [15]:
# * степень полинома (для ядра 'poly') 
gs.best_estimator_.get_params()['svc__degree']

3

---

📚 **Пример визуализации небольшого дерева**

Лучшее дерево с обрезкой по-прежнему слишком велико для визуализации. Для примера нарисуем одно из небольших деревьев с обрезкой и выведем его же в виде текста.  
Посмотреть результаты сеточного поиска можно в объектах:
* `gs.cv_results_['params']` – список сочетаний параметров;
* `gs.cv_results_['mean_test_score']` – значения для сочетаний параметров
(средние по блокам перекрёстной проверки).
---

In [16]:
# записываем точность
score.append(np.around(gs.best_score_, 3)) 
score_models.append('sc_pca_svc') 
 
print('Acc с перекрёстной проверкой', 
      '\nдля модели', score_models[1], ':', score[1]) 

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 0.816


# Прогноз на отложенные наблюдения по лучшей модели

В данном случае модель SVM показывает большую точность, чем модель логистической регрессии.

In [17]:
# сводка по точности моделей 
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,sc_pca_logit,0.798
1,sc_pca_svc,0.816


Сделаем прогноз на отложенные наблюдения с помощью второго ансамбля.   

In [28]:
# прогноз с помощью лучшей модели ансамбля с SVC 
y_hat = gs.best_estimator_.predict(X=DF_predict.iloc[:, 0:23]) 
 
# точность 
# характеристики точности 
print(classification_report(DF_predict.iloc[:, 23:], y_hat)) 

              precision    recall  f1-score   support

           0       0.82      0.96      0.88      2292
           1       0.71      0.30      0.42       708

    accuracy                           0.81      3000
   macro avg       0.76      0.63      0.65      3000
weighted avg       0.79      0.81      0.77      3000

